<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Buffer - Send text to Idea

**Tags:** #buffer #idea #text #send #python #api #command

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-10-11 (Created: 2023-10-11)

**Description:** This notebook sent a text to a Buffer Idea.

**References:**
- [Buffer - Create Idea](https://publish.buffer.com/content)

## Input

### Import libraries

In [ ]:
import requests
import json
import naas
import pydash

### Setup variables
- `organization_id`: Buffer organization ID
- `text`: This variable contains the text content that you want to post on the Buffer profile. You can modify this string to change the content of the post.
- `body`: This variable stores the body to be send by the webhook.

In [ ]:
organization_id = naas.secret.get("BUFFER_ORGANIZATION")
text = "My Buffer Idea"
body = {}

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [ ]:
# Parameters
if len(body) > 0:
    organization_id = body.get("organization_id")
    text = body.get("text")

## Model

### Send text to Idea

This function will send text to Idea using the Buffer API.

In [ ]:
def parse_cookie(cookie):
    data = {}
    for c in cookie.split(";"):
        key = c.split("=", 1)[0].strip()
        value = c.split("=", 1)[-1].strip()
        data[key] = value
    return data

def send_text_to_idea(organization_id, text):
    # Init
    status = "ok"
    try:
        result = None
        cookie = naas.secret.get("BUFFER_COOKIES")
        cookies = parse_cookie(cookie)

        # Base URL
        url = "https://graph.buffer.com/"

        # Headers
        headers = {
            "authority": "graph.buffer.com",
            "accept": "*/*",
            "accept-language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
            "content-type": "application/json",
            "origin": "https://publish.buffer.com",
            "referer": "https://publish.buffer.com/",
            "sec-ch-ua": '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"Windows"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-site",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
            "x-buffer-client-id": "webapp-publishing"
        }

        data = {
            "operationName": "CreateIdea",
            "variables": {
                "input": {
                    "content": {
                        "text": text,
                        "media": [],
                        "tags": []
                    },
                    "organizationId": organization_id,
                    "source": "content"
                }
            },
            "query": 'mutation CreateIdea($input: IdeaCreationInput\u0021) {\n  ideaCreate(input: $input) {\n    ... on IdeaMutationSuccess {\n      success\n      message\n      idea {\n        id\n        content {\n          text\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    ... on CoreWebAppCommonError {\n      message\n      code\n      error\n      __typename\n    }\n    __typename\n  }\n}\n'
        }
        res = requests.post(url, headers=headers, data=json.dumps(data), cookies=cookies)
        print(res)
        if res.status_code == 200:
            message = pydash.get(res.json(), "data.ideaCreate.message")
        else:
            print(res.status_code)
            print(res.text)
            message = res.text
    except Exception as e:
            message = f"Template error: {e}"
    return status, message

status, message = send_text_to_idea(organization_id, text)

## Output

### Create or update Webhook

In [ ]:
naas.webhook.add()

### Return JSON response
Response sent to the browser before displayed in Chat UI.

In [ ]:
naas.webhook.respond_json(
    {
        "status": status, 
        "message": message
    }
)